In [ ]:
# Load imports and configuration

%load_ext autoreload
%autoreload 2

import hydra
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()
hydra.initialize(version_base=None, config_path="../conf")
cfg = hydra.compose(config_name="config")

tickers = cfg.market.tickers
interval = cfg.market.interval
fee_rate = cfg.market.fee_rate
initial_cash = cfg.market.initial_cash
risk_free_rate = cfg.market.risk_free_rate_annual

from skopt.space import Integer, Real

from modules.performance.strategy import Strategy
from modules.data_services.data_loaders import load_data
from modules.performance.statistical_tests import engle_granger_cointegration
from modules.data_services.data_utils import load_btc_benchmark, save_to_parquet
from modules.visualization.plots import plot_pnl, plot_positions, plot_zscore

In [ ]:
pair_selection_start = "2024-01-01"
pair_selection_end = "2024-03-01"

In [ ]:
# Load data and perform pair selection

ps_df = load_data(
    tickers=tickers,
    start=pair_selection_start,
    end=pair_selection_end,
    interval=interval
)

eg_df = engle_granger_cointegration(
    df=ps_df,
    source="log_prices"
)

eg_df.head(5)

In [ ]:
# Select TOP1 pair

tickers = eg_df.iloc[0:1, 0].tolist()
ticker_x = tickers[0].split('-')[0]
ticker_y = tickers[0].split('-')[1]

print(f"Selected pair: {ticker_x}/{ticker_y}")

In [ ]:
opt_pre_start = "2024-01-01"
opt_start = "2024-02-01"
opt_end = "2024-03-01"

test_beta_calculation_start = "2024-01-01"
test_pre_start = "2024-02-01"
test_start = "2024-03-01"
test_end = "2024-04-01"

In [ ]:
# Set up a Strategy

bt = Strategy(
    ticker_x=ticker_x,
    ticker_y=ticker_y,
    start=test_beta_calculation_start,
    end=test_end,
    interval=interval,
    fee_rate=fee_rate,
    initial_cash=initial_cash,
    risk_free_rate_annual=risk_free_rate,
    beta_hedge="static_hedge",
    source="log",
)

In [ ]:
# Run optimization

static_params = {
    "stop_loss": 10  # Disable SL
}

param_space = [
    Integer(10, 400, name="rolling_window"),
    Real(1.01, 4.00, name="entry_threshold"),
    Real(0.0, 1.00, name="exit_threshold"),
    # Real(1.01, 3.00, name="stop_loss"),
]

metric = ("sortino_annual_with_trades", "net")

best_params, best_score = bt.run_optimization(
    opt_start=opt_start,
    opt_end=opt_end,
    static_params=static_params,
    param_space=param_space,
    metric = metric,
)

print(best_params, best_score)

In [ ]:
entry_threshold = best_params["entry_threshold"]
exit_threshold = best_params["exit_threshold"]
# stop_loss = best_params["stop_loss"]
stop_loss = 10
rolling_window = best_params["rolling_window"]

In [ ]:
# Run strategy

result = bt.run_strategy(
    rolling_window=rolling_window,
    entry_threshold=entry_threshold,
    exit_threshold=exit_threshold,
    stop_loss=stop_loss,
    test_start=test_start,
    test_end=test_end,
)
save_to_parquet(df=result.data, file_name="result_data")

In [ ]:
# Show data

result.data

In [ ]:
# Show statistics

result.stats

In [ ]:
# Show plots

plot_positions(result, directory="top1_test", save=True, show=True)
btc_data = load_btc_benchmark(
    test_start=test_start,
    test_end=test_end,
    interval=interval,
)
plot_pnl(result, btc_data, directory="top1_test", save=True, show=True)
plot_zscore(result, directory="top1_test", save=True, show=True)